In [17]:
import pandas as pd
import numpy 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
#load dataset with pandas

In [3]:
from os.path import expanduser as ospath
Variant_1 = pd.read_csv(ospath("path/to/file/Features_Variant_1.csv"),header=None)
Variant_2 = pd.read_csv(ospath("path/to/file/Features_Variant_2.csv"),header=None)
Variant_3 = pd.read_csv(ospath("path/to/file/Features_Variant_3.csv"),header=None)
Variant_4 = pd.read_csv(ospath("path/to/file/Features_Variant_4.csv"),header=None)
Variant_5 = pd.read_csv(ospath("path/to/file/Features_Variant_5.csv"),header=None)
TestSet = pd.read_csv(ospath("path/to/file/Features_TestSet.csv"),header=None)

In [4]:
#header file for all dtaset header info
header = pd.read_csv(ospath("path/to/file/header.csv"),header=0)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
#Preprocess dataset,define function for feature/target separation, train/test split
def preprocessing(variant_x):
    train_variant = pd.DataFrame(variant_x)
    train_variant.columns=header.columns
    target = 'Target'
    features = list(train_variant.columns)
    features = [f for f in features if f!=target]
    X=train_variant[features]
    y=train_variant[[target]]
    X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.3,random_state=42)
    return X_train_1, X_test_1, y_train_1, y_test_1

In [7]:
X_train_1, X_test_1, y_train_1, y_test_1 =preprocessing(Variant_3)

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
#using random forest regressor conduct prediction 

In [15]:
#function for Grid_search_cv for randomforestregressor
def Grid_Search_CV_RFR(X_train, y_train):
    estimator = RandomForestRegressor()
    param_grid = { 
            "n_estimators"      : [10,20,30],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False],
            }

    grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=3,scoring='mean_squared_error')

    grid.fit(X_train, y_train)

    return grid.best_score_ , grid.best_params_

In [13]:
best_score, best_params= Grid_Search_CV_RFR(X_train_1, y_train_1)

C:\Users\chippy\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  best_estimator.fit(X, y, **self.fit_params)


In [14]:
best_score, best_params

(-339.38714677290852,
 {'bootstrap': True,
  'max_features': 'auto',
  'min_samples_split': 4,
  'n_estimators': 30})

In [15]:
#The baseline model-elevate test result on each variant_x dataset

In [16]:
from sklearn.metrics import r2_score

In [17]:
def RFR(X_train, X_test, y_train, y_test, best_params):
    from sklearn.ensemble import RandomForestRegressor
    estimator = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
    estimator.fit(X_train,y_train)
    y_predict = estimator.predict(X_test)
    print ("R2 score:",r2_score(y_test,y_predict))
    return y_test,y_predict

In [18]:
RFR(X_train_1,X_test_1, y_train_1, y_test_1,best_params)

C:\Users\chippy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


R2 score: 0.706359694969


(        Target
 36793        0
 24455       24
 20088      192
 96778        0
 104317       1
 54272        1
 42138       10
 34917        3
 32434       11
 39903        0
 59110        0
 19716        2
 949          0
 36858        0
 83649        0
 105151       0
 14766       11
 53470        5
 52731        0
 82200        7
 201          0
 13821        2
 69417        0
 22775        0
 36206        0
 11241        4
 113101       0
 93170        5
 81290        2
 78327        0
 ...        ...
 52405        0
 119151       0
 35463        1
 179          0
 37398        0
 21338        0
 36663        0
 64846        0
 56604        0
 85188        2
 30258        1
 82492        1
 107100       0
 16156        6
 96744        0
 90210        0
 109323       3
 6237         0
 37303       44
 71929       12
 21821       22
 13676        0
 2295        36
 8898         0
 35882        0
 100077       2
 38483        0
 98945        1
 95120        0
 91201        0
 
 [3633

In [ ]:
#scalling X_train,X_test-Y rmse change

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_1)
X_trainS = scaler.transform(X_train_1)
X_testS = scaler.transform(X_test_1)

In [21]:
best_score, best_params= Grid_Search_CV_RFR(X_trainS, y_train_1)

C:\Users\chippy\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  best_estimator.fit(X, y, **self.fit_params)


In [22]:
best_score, best_params

(-342.46631610687069,
 {'bootstrap': True,
  'max_features': 'auto',
  'min_samples_split': 2,
  'n_estimators': 30})

In [23]:
RFR(X_trainS,X_testS, y_train_1, y_test_1,best_params)

C:\Users\chippy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


R2 score: 0.720385159732


(        Target
 36793        0
 24455       24
 20088      192
 96778        0
 104317       1
 54272        1
 42138       10
 34917        3
 32434       11
 39903        0
 59110        0
 19716        2
 949          0
 36858        0
 83649        0
 105151       0
 14766       11
 53470        5
 52731        0
 82200        7
 201          0
 13821        2
 69417        0
 22775        0
 36206        0
 11241        4
 113101       0
 93170        5
 81290        2
 78327        0
 ...        ...
 52405        0
 119151       0
 35463        1
 179          0
 37398        0
 21338        0
 36663        0
 64846        0
 56604        0
 85188        2
 30258        1
 82492        1
 107100       0
 16156        6
 96744        0
 90210        0
 109323       3
 6237         0
 37303       44
 71929       12
 21821       22
 13676        0
 2295        36
 8898         0
 35882        0
 100077       2
 38483        0
 98945        1
 95120        0
 91201        0
 
 [3633

In [ ]:
#noted some imporvement after scalling the data

In [ ]:
#Keras+gridsearch

In [8]:
# define base model
def baseline_model():
# create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
# Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [13]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
model = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [ ]:
param_grid = dict(epochs=[10,20,30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_1,y_train_1)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#for loop i in dataset[Variant_1,Variant_2...]

In [ ]:
# https://stackoverflow.com/questions/43590489/gridsearchcv-random-forest-regressor-tuning-best-params
#for looping datasets

In [ ]:
#conclusion